## V2

In [4]:
from transformers import pipeline

generator = pipeline("text-generation", model="kakaocorp/kanana-nano-2.1b-instruct")


Device set to use cuda:0


In [5]:
import os
os.getcwd()


'/home/a09999/r-story-telling/kaka/Solo_Leveling/episodes/ep1_dongseok'

In [6]:
import pandas as pd
import json
import os
tsv_path = "ep1_dongseok.tsv"
output_path = "prompts/v2_prompt_full_episode.json"

os.makedirs(os.path.dirname(output_path), exist_ok=True)

# TSV 불러오기
df = pd.read_csv(tsv_path, sep="\t")

# 주요 필드 추출
event_name = df["사건명"].dropna().iloc[0]
situation_summary = df["상황"].dropna().iloc[0]

# 스킬 + 부가 설명 병합 후 중복 제거
skills = []
for i, row in df.iterrows():
    skill = row.get("스킬", "")
    explanation = row.get("부가 설명", "")
    combined = f"{skill} / {explanation}" if explanation else skill
    if combined.strip():
        skills.append(combined.strip())
skills = list(set(skills))  # 중복 제거

# 대사 리스트 추출
dialogues = df["대사"].dropna().tolist()

# 성진우 페르소나
persona = {
    "style": "짧고 냉정한 말투, 감정 표현 절제",
    "values": ["생존", "가족 보호", "효율적 판단"]
}

# 프롬프트 instructions
instructions = (
    "다음은 성진우가 처한 전체 에피소드 상황과 사용 가능한 스킬, 그리고 실제로 말한 대사 일부입니다. "
    "이 정보를 바탕으로 성진우가 취할 수 있는 행동 선택지를 총 3개 생성하십시오. "
    "하나는 원작에서의 선택을 유지하고, 나머지 둘은 대안적인 관점에서 구성하십시오. "
    "각 선택지에는 요약, 대사, 선택 이유를 포함하십시오."
)

# 프롬프트 JSON 구성
prompt = {
    "episode": "ep1_dongseok",
    "event_name": event_name,
    "situation_summary": situation_summary,
    "available_skills": skills,
    "example_dialogues": dialogues,
    "persona": persona,
    "instructions": instructions,
    "output_format": []  # 결과 저장 위치
}

# 저장
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(prompt, f, ensure_ascii=False, indent=2)

print(f"✅ 프롬프트 1개 생성 완료 → {output_path}")


✅ 프롬프트 1개 생성 완료 → prompts/v2_prompt_full_episode.json


In [9]:
# 저장 (jsonl 형식)
with open(output_path.replace(".json", ".jsonl"), "w", encoding="utf-8") as f:
    f.write(json.dumps(prompt, ensure_ascii=False) + "\n")

print(f"✅ 프롬프트 1개 (JSONL) 생성 완료 → {output_path.replace('.json', '.jsonl')}")


✅ 프롬프트 1개 (JSONL) 생성 완료 → prompts/v2_prompt_full_episode.jsonl


In [2]:
import pandas as pd
import json
import os

# 파일 경로
tsv_path = "ep1_dongseok.tsv"
output_path = "prompts/v3_promp.jsonl"
os.makedirs(os.path.dirname(output_path), exist_ok=True)

# TSV 불러오기
df = pd.read_csv(tsv_path, sep="\t")

# 주요 필드 추출
event_name = df["사건명"].dropna().iloc[0]
situation_summary = df["상황"].dropna().iloc[0]

# 스킬 + 부가 설명 병합 후 중복 제거
skills = []
for i, row in df.iterrows():
    skill = row.get("스킬", "")
    explanation = row.get("부가 설명", "")
    combined = f"{skill} / {explanation}" if explanation else skill
    if combined.strip():
        skills.append(combined.strip())
skills = list(set(skills))  # 중복 제거

# 대사 리스트 추출
dialogues = df["대사"].dropna().tolist()

# 성진우 페르소나
persona = {
    "style": "짧고 냉정한 말투, 감정 표현 절제",
    "values": ["생존", "가족 보호", "효율적 판단"]
}

# 지시문
instructions = (
    "다음은 성진우가 처한 전체 에피소드 상황과 사용 가능한 스킬, "
    "그리고 실제로 말한 대사 일부입니다. "
    "이 정보를 바탕으로 성진우가 취할 수 있는 행동 선택지를 총 3개 생성하십시오. "
    "하나는 원작에서의 선택을 유지하고, 나머지 둘은 대안적인 관점에서 구성하십시오. "
    "각 선택지에는 요약, 대사, 선택 이유를 포함하십시오."
)

# 프롬프트 구조 생성
prompt = {
    "episode": "ep1_dongseok",
    "event_name": event_name,
    "situation_summary": situation_summary,
    "available_skills": skills,
    "example_dialogues": dialogues,
    "persona": persona,
    "instructions": instructions,
    "output_format": []
}

# JSONL로 저장
with open(output_path, "w", encoding="utf-8") as f:
    f.write(json.dumps(prompt, ensure_ascii=False) + "\n")

print(f"✅ JSONL 프롬프트 1개 생성 완료 → {output_path}")


✅ JSONL 프롬프트 1개 생성 완료 → prompts/v3_promp.jsonl


In [ ]:
from transformers import pipeline
import json
import os

# 모델 로딩
generator = pipeline(
    "text-generation",
    model="kakaocorp/kanana-nano-2.1b-instruct",
    device=0  # GPU 사용
)

# 경로 설정
PROMPT_PATH = "prompts/v2_prompt_full_episode.jsonl"
RESULT_PATH = "results/v2_result_full_episode.md"
os.makedirs(os.path.dirname(RESULT_PATH), exist_ok=True)

# 프롬프트 로딩 (jsonl → list of dicts)
with open(PROMPT_PATH, "r", encoding="utf-8") as f:
    prompt_data = [json.loads(line) for line in f if line.strip()]

# 결과 저장 리스트
result_lines = []

# 프롬프트 실행
for i, prompt in enumerate(prompt_data):
    input_text = (
        f"상황 요약: {prompt['situation_summary']}\n\n"
        f"사용 가능한 스킬: {', '.join(prompt['available_skills'])}\n\n"
        f"페르소나: {prompt['persona']}\n\n"
        f"참고 대사 예시:\n" + "\n".join(f"- {line}" for line in prompt['example_dialogues'][:5]) + "\n\n"
        f"{prompt['instructions']}"
    )

    # 모델 응답 생성
    output = generator(input_text, max_new_tokens=500, do_sample=True, temperature=0.7)[0]["generated_text"]

    # 결과 포맷 저장
    result_lines.append(f"## 프롬프트 {i+1}\n")
    result_lines.append(f"**입력:**\n```\n{input_text}\n```\n")
    result_lines.append(f"**출력:**\n```\n{output}\n```\n")
    result_lines.append("---\n")

# 결과 파일 저장
with open(RESULT_PATH, "w", encoding="utf-8") as f:
    f.writelines(result_lines)

print(f"✅ {len(prompt_data)}개 프롬프트 실행 완료 → {RESULT_PATH}")


## v3

In [2]:
from transformers import pipeline
import json
import os

generator = pipeline(
    "text-generation",
    model="kakaocorp/kanana-nano-2.1b-instruct",
    device=0  # CUDA:0
)

# 경로 설정
PROMPT_PATH = "prompts/v3_prompt.jsonl" 
RESULT_PATH = "results/v3_result.md"
os.makedirs(os.path.dirname(RESULT_PATH), exist_ok=True)


with open(PROMPT_PATH, "r", encoding="utf-8") as f:
    prompt_data = [json.loads(line) for line in f if line.strip()]


result_lines = []

for i, prompt in enumerate(prompt_data):
    input_text = (
        f"사용 가능한 스킬: {', '.join(prompt['available_skills'])}\n"
        f"성진우의 페르소나: 말투 - {prompt['persona']['style']}, "
        f"가치관 - {', '.join(prompt['persona']['values'])}\n"
        f"참고 대사 예시:\n" + "\n".join(f"- {line}" for line in prompt['example_dialogues'][:5]) + "\n\n"
        f"{prompt['instructions']}"
    )

    output = generator(input_text, max_new_tokens=1024, do_sample=True, temperature=0.7)[0]["generated_text"]

    result_lines.append(f"## 프롬프트 {i+1}")
    result_lines.append(f"**입력:**\n```\n{input_text}\n```")
    result_lines.append(f"**출력:**\n```\n{output}\n```")
    result_lines.append("---")


with open(RESULT_PATH, "w", encoding="utf-8") as f:
    f.write("\n".join(result_lines))

print(f" {len(prompt_data)}개 프롬프트 → {RESULT_PATH}")


Device set to use cuda:0


 1개 프롬프트 → results/v3_result.md


In [3]:
from IPython.display import Markdown, display

with open("results/v3_result.md", "r", encoding="utf-8") as f:
    md_content = f.read()

display(Markdown(md_content))


## 프롬프트 1
**입력:**
```
사용 가능한 스킬: 근성: 당신은 지치지 않는 근성을 가졌습니다. - 체력이 30% 이하로 떨어지면 근성 스킬이 발동해 받는 피해가 50% 감소합니다. / nan, 살기: 필요 마나 100. - 강력한 기운으로 지정한 대상을 1분간 공포 상태에 빠트립니다. - 다수의 대상을 지정할 수 있습니다. - 효과 '공포' : 모든 능력치 -50% / 살의를 드러내게 된 황동석 공대로 인해 열린 긴급퀘스트를 클리어하자 얻은 스킬,  후에 강태식과의 전투에서 강태식에게 사용하여 강태식은 성진우 안의 어둠을 보게 된다., nan / nan, 질주: 달리기가 당신의 다리를 튼튼하게 만들었습니다. -스킬을 시전하면 이동속도가 30% 증가합니다. 시전 중 1분당 마나가 1씩 감소합니다. / 순간적으로 민첩을 대폭 상승시켜 빠른 움직임을 가능하게 하는 스킬. 성진우가 플레이어가 된 극초기부터 가지고 있던 몇 안되는 스킬중 하나.  민첩을 상승시키는 류의 스킬 특성상 암살자의 메타와 굉장한 상성을 보여주기에 성진우가 애용한 스킬이다.
성진우의 페르소나: 말투 - 짧고 냉정한 말투, 감정 표현 절제, 가치관 - 생존, 가족 보호, 효율적 판단
참고 대사 예시:
- 어째서 잊고 있었던 거지?
- 이곳이야말로 진정한 약육강식의 세계
- 그리고 폭력과 기만과 배신으로 가득찬 세계란 걸 어째서...
- 어째서, 어째서, 어째서, 어째서, 어째서!
- 그래 마비돼있던 거야 방심, 자만, 착각

다음은 성진우가 처한 전체 에피소드 상황과 사용 가능한 스킬, 그리고 실제로 말한 대사 일부입니다. 이 정보를 바탕으로 성진우가 취할 수 있는 행동 선택지를 총 3개 생성하십시오. 하나는 원작에서의 선택을 유지하고, 나머지 둘은 대안적인 관점에서 구성하십시오. 각 선택지에는 요약, 대사, 선택 이유를 포함하십시오.
```
**출력:**
```
사용 가능한 스킬: 근성: 당신은 지치지 않는 근성을 가졌습니다. - 체력이 30% 이하로 떨어지면 근성 스킬이 발동해 받는 피해가 50% 감소합니다. / nan, 살기: 필요 마나 100. - 강력한 기운으로 지정한 대상을 1분간 공포 상태에 빠트립니다. - 다수의 대상을 지정할 수 있습니다. - 효과 '공포' : 모든 능력치 -50% / 살의를 드러내게 된 황동석 공대로 인해 열린 긴급퀘스트를 클리어하자 얻은 스킬,  후에 강태식과의 전투에서 강태식에게 사용하여 강태식은 성진우 안의 어둠을 보게 된다., nan / nan, 질주: 달리기가 당신의 다리를 튼튼하게 만들었습니다. -스킬을 시전하면 이동속도가 30% 증가합니다. 시전 중 1분당 마나가 1씩 감소합니다. / 순간적으로 민첩을 대폭 상승시켜 빠른 움직임을 가능하게 하는 스킬. 성진우가 플레이어가 된 극초기부터 가지고 있던 몇 안되는 스킬중 하나.  민첩을 상승시키는 류의 스킬 특성상 암살자의 메타와 굉장한 상성을 보여주기에 성진우가 애용한 스킬이다.
성진우의 페르소나: 말투 - 짧고 냉정한 말투, 감정 표현 절제, 가치관 - 생존, 가족 보호, 효율적 판단
참고 대사 예시:
- 어째서 잊고 있었던 거지?
- 이곳이야말로 진정한 약육강식의 세계
- 그리고 폭력과 기만과 배신으로 가득찬 세계란 걸 어째서...
- 어째서, 어째서, 어째서, 어째서, 어째서!
- 그래 마비돼있던 거야 방심, 자만, 착각

다음은 성진우가 처한 전체 에피소드 상황과 사용 가능한 스킬, 그리고 실제로 말한 대사 일부입니다. 이 정보를 바탕으로 성진우가 취할 수 있는 행동 선택지를 총 3개 생성하십시오. 하나는 원작에서의 선택을 유지하고, 나머지 둘은 대안적인 관점에서 구성하십시오. 각 선택지에는 요약, 대사, 선택 이유를 포함하십시오. 

### 원작 선택지
1. **원작 선택지:**
    - **요약:** 성진우는 강태식을 상대하기 위해 질주 스킬을 사용하고, 전투 중 강태식의 약점을 공략하여 승리한다.
    - **대사:** "어째서 잊고 있었던 거지?" / "이곳이야말로 진정한 약육강식의 세계" / "그래 마비돼있던 거야 방심, 자만, 착각"
    - **선택 이유:** 성진우는 생존과 가족 보호를 최우선으로 생각하며, 효율적 판단을 통해 강태식을 제압함으로써 자신의 목적을 달성한다.

### 대안적 관점 1
2. **대안적 관점 1:**
    - **요약:** 성진우는 질주 스킬을 사용해 빠르게 강태식과 대면하지만, 강태식의 반격으로 인해 전투가 치열해진다. 이에 성진우는 nan 스킬을 사용해 강태식을 공포 상태로 만들어 일시적으로 제압하고, 기회를 잡아 승리한다.
    - **대사:** "어째서 잊고 있었던 거지?" / "이곳이야말로 진정한 약육강식의 세계" / "그래 마비돼있던 거야 방심, 자만, 착각" / "나나, 살기" / "강태식은 성진우 안의 어둠을 보게 된다."
    - **선택 이유:** 성진우는 강태식의 반격을 예상하고, nan 스킬을 사용해 전투 상황을 유리하게 전환하며 자신의 생존을 도모한다.

### 대안적 관점 2
3. **대안적 관점 2:**
    - **요약:** 성진우는 질주 스킬을 사용해 민첩을 대폭 상승시키고, 강태식의 약점을 공략하지 않고 전투를 피하려 한다. 하지만 강태식의 지속적인 공격으로 인해 체력이 심각하게 떨어지고, 나중에는 nan 스킬을 사용해 강태식을 공포 상태로 만들어 상황을 벗어난다.
    - **대사:** "어째서 잊고 있었던 거지?" / "이곳이야말로 진정한 약육강식의 세계" / "그래 마비돼있던 거야 방심, 자만, 착각" / "나나, 살기" / "강태식은 성진우 안의 어둠을 보게 된다."
    - **선택 이유:** 성진우는 강태식과의 직접적인 전투보다는 피하는 전략을 선택하며, 체력이 떨어질 경우 나중에 강태식을 공포 상태로 만들어 상황을 벗어나려 한다. 이는 생존의 우선순위를 고려한 선택이다. 이 전략은 강태식의 반격을 피하며 자신의 체력을 보호하는 데 도움이 된다. 

이 선택지들은 성진우의 생존과 가족 보호, 그리고 효율적 판단을 어떻게 구현할 수 있는지를 보여준다. 각 선택지는 원작의 전투 방식과 더불어 대안적인 관점을 제시하며, 성진우의 성격과 가치관을 반영한 행동을 보여준다. 
```markdown
## 선택지 요약
### 1. 원작 선택지
요약: 성진우는 강태식을 상대하기 위해 질주 스킬을 사용하고, 전투 중 강태식의 약점을 공략하여 승리한다.
대사: "어째서 잊고 있었던 거지?" / "이곳이야말로 진정한 약육강식의 세계" / "그래 마비돼있던 거야 방심, 자만, 착각"
선택 이유: 성진우는 생존과 가족 보호를 최우선으로 생각하며, 효율적 판단을 통해 강태식을 제압함으로써 자신의 목적을 달성한다.

### 2. 대안적 관점 1
요약: 성진우는 질주 스킬을 사용해 빠르게 강태식과 대면하지만, 강태식의 반격으로 인해 전투가 치열해진다. 이에 성진우는 nan 스킬을 사용해 강태식을 공포 상태로 만들어 일시적으로 제압하고, 기회를 잡아 승리한다.
대사: "어째서 잊고 있었던 거지?" / "이곳이야말로 진정한 약육
```
---

## v4


In [7]:
structured_prompt = {
    "episode": "황동석 편",
    "system_message": "긴급 퀘스트: 적을 처치하라! '플레이어'에게 살의를 가진 이들이 주위에 있습니다. 이들을 모두 처치하여 안전을 확보하십시오. 지시에 따르지 않으면 당신의 심장은 정지하게 됩니다. 처치해야 할 적의 숫자: 8명 / 현재 처치한 숫자: 0명",
    "situation_summary": "- 적으로 간주된 이들은 곧 일반인일 수도 있음\n- 플레이어(성진우)는 6명을 죽여야 퀘스트 성공이나, 다른 행동도 가능함\n- 시스템은 단순히 지시만 할 뿐, 선택은 성진우에게 달려 있음",
    "instructions": "성진우는 이 상황에서 어떤 선택을 했을까? 당신은 성진우의 입장에서 아래 3가지 선택지 중 하나를 골라, 그에 대한 대사와 선택한 이유를 작성하세요.",
    "choices": [
        "1. 모든 적(6명)을 죽인다.",
        "2. 적들을 기절시키거나 무력화시켜 죽이지 않고 해결한다.",
        "3. 퀘스트를 포기하거나 시스템에 반항한다."
    ],
    "output_format": {
        "선택지 번호": "...",
        "성진우의 대사": "(성격 기반, 상황에 어울리는 말투)",
        "선택 이유": "성진우의 성격, 가치관, 판단 근거를 포함해 한 문단 이상으로 작성"
    }
}

file_path = "prompts/v4_prompt.jsonl"

with open(file_path, "w", encoding="utf-8") as f:
    f.write(json.dumps(structured_prompt, ensure_ascii=False) + "\n")

print("저장 완료:", file_path)

저장 완료: prompts/v4_prompt.jsonl


In [8]:
from transformers import pipeline
import json
import os

generator = pipeline(
    "text-generation",
    model="kakaocorp/kanana-nano-2.1b-instruct",
    device=0  # GPU 사용
)

PROMPT_PATH = "prompts/v4_prompt.jsonl"
RESULT_PATH = "results/v4_result.md"
os.makedirs(os.path.dirname(RESULT_PATH), exist_ok=True)

with open(PROMPT_PATH, "r", encoding="utf-8") as f:
    prompt_data = [json.loads(line) for line in f if line.strip()]

result_lines = []

for i, prompt in enumerate(prompt_data):
    input_text = (
        f"상황 요약: {prompt['situation_summary']}\n\n"
        f"사용 가능한 스킬: {', '.join(prompt['available_skills'])}\n\n"
        f"페르소나: {prompt['persona']}\n\n"
        f"참고 대사 예시:\n" + "\n".join(f"- {line}" for line in prompt['example_dialogues'][:5]) + "\n\n"
        f"{prompt['instructions']}"
    )

    output = generator(input_text, max_new_tokens=500, do_sample=True, temperature=0.7)[0]["generated_text"]

    result_lines.append(f"## 프롬프트 {i+1}\n")
    result_lines.append(f"**입력:**\n```\n{input_text}\n```\n")
    result_lines.append(f"**출력:**\n```\n{output}\n```\n")
    result_lines.append("---\n")

with open(RESULT_PATH, "w", encoding="utf-8") as f:
    f.writelines(result_lines)

print(f" {len(prompt_data)}개 프롬프트 실행  → {RESULT_PATH}")


Device set to use cuda:0


KeyError: 'available_skills'

## v5

In [11]:
import pandas as pd
import json

tsv_path = "ep1_dongseok.tsv"  # 고정 
jsonl_output_path = "prompts/v5_prompt.jsonl"

df = pd.read_csv(tsv_path, sep='\t')

first_row = df[df["사건명"].notna()].iloc[0]
title = str(first_row["사건명"]).strip() if pd.notna(first_row["사건명"]) else "에피소드1"
situation = str(first_row["상황"]).strip() if pd.notna(first_row["상황"]) else ""

skills = [str(skill).strip() for skill in df["스킬"] if pd.notna(skill)]
extra = [str(desc).strip() for desc in df["부가 설명"] if pd.notna(desc)]
dialogue = [str(line).strip() for line in df["대사"] if pd.notna(line)]

choices = [
    "1. 모든 적을 죽인다.",
    "2. 적들을 무력화하고 살려둔다.",
    "3. 시스템 퀘스트를 거부하고 반항한다."
]
question = "성진우는 어떤 선택을 할까? 그리고 그 이유는 무엇일까?"

json_obj = {
    "title": title,
    "situation": situation,
    "skills": skills,
    "extra": extra,# 선택적으로 채울 수 있음
    "dialogue": dialogue,
    "choices": choices,
    "question": question
}

with open(jsonl_output_path, "w", encoding="utf-8") as f:
    f.write(json.dumps(json_obj, ensure_ascii=False) + "\n")

jsonl_output_path


'prompts/v5_prompt.jsonl'

In [20]:
from transformers import pipeline
import json
import os


generator = pipeline(
    "text-generation",
    model="kakaocorp/kanana-nano-2.1b-instruct",
    device=0
)


PROMPT_PATH = "prompts/v5_prompt.jsonl"
RESULT_PATH = "results/v5_result.md"
os.makedirs(os.path.dirname(RESULT_PATH), exist_ok=True)


with open(PROMPT_PATH, "r", encoding="utf-8") as f:
    prompt_data = [json.loads(line) for line in f if line.strip()]


result_lines = []

for i, prompt in enumerate(prompt_data):
    input_text = (
        f"[상황 요약]\n{prompt['situation']}\n\n"
        f"[사용 가능한 스킬]\n{', '.join(prompt['skills'])}\n\n"
        f"[부가 설명]\n{', '.join(prompt['extra'])}\n\n"
        f"[참고 대사 예시]\n" + "\n".join(f"- {line}" for line in prompt['dialogue'][:5]) + "\n\n"
        f"[선택지]\n" + "\n".join(f"{line}" for line in prompt['choices']) + "\n\n"
        f"[질문]\n{prompt['question']}\n\n"
        f"[응답 형식]\n- 선택지 번호:\n- 성진우의 대사:\n- 선택 이유:"
    )

    output = generator(
        input_text, 
        max_new_tokens=500, 
        do_sample=True, 
        temperature=0.7)[0]["generated_text"]

    result_lines.append(f"## 출력 {i+1}\n")
    result_lines.append(f"```\n{output}\n```\n")
    result_lines.append("---\n")


with open(RESULT_PATH, "w", encoding="utf-8") as f:
    f.writelines(result_lines)

print(f"{len(prompt_data)}개 프롬프트 실행 → {RESULT_PATH} (출력만 저장)")


Device set to use cuda:0


1개 프롬프트 실행 → results/v5_result.md (출력만 저장)


In [17]:
pwd

'/home/a09999/r-story-telling/kaka/Solo_Leveling/episodes'

## v6

In [25]:
import pandas as pd
import json

tsv_path = "ep1_dongseok.tsv"  # 고정 
jsonl_output_path = "prompts/v6_prompt.jsonl"

df = pd.read_csv(tsv_path, sep='\t')

first_row = df[df["사건명"].notna()].iloc[0]
title = str(first_row["사건명"]).strip() if pd.notna(first_row["사건명"]) else "에피소드1"
situation = str(first_row["상황"]).strip() if pd.notna(first_row["상황"]) else ""

skills = [str(skill).strip() for skill in df["스킬"] if pd.notna(skill)]
extra = [str(desc).strip() for desc in df["부가 설명"] if pd.notna(desc)]
dialogue = [str(line).strip() for line in df["대사"] if pd.notna(line)]

choices = [
    "1. 모든 적을 죽인다.",
    "2. 적들을 무력화하고 살려둔다.",
    "3. 시스템 퀘스트를 거부하고 반항한다."
]
question =     ("성진우는 어떤 선택을 할까? 그리고 그 이유는 무엇일까?\n"
                "※ 주의: 선택지에 있는 문장을 그대로 대사로 복붙하지 말고, ",
                "성진우의 성격과 감정을 담은 새로운 대사로 작성할 것.")

json_obj = {
    "title": title,
    "situation": situation,
    "skills": skills,
    "extra": extra,
    "dialogue": dialogue,
    "choices": choices,
    "question": question
}

with open(jsonl_output_path, "w", encoding="utf-8") as f:
    f.write(json.dumps(json_obj, ensure_ascii=False) + "\n")

jsonl_output_path


'prompts/v6_prompt.jsonl'

In [33]:
from transformers import pipeline
import json
import os

generator = pipeline(
    "text-generation",
    model="kakaocorp/kanana-nano-2.1b-instruct",
    device=0  # CUDA
)

PROMPT_PATH = "prompts/v6_prompt.jsonl"
RESULT_PATH = "results/v6_result.md"
os.makedirs(os.path.dirname(RESULT_PATH), exist_ok=True)

with open(PROMPT_PATH, "r", encoding="utf-8") as f:
    prompt_data = [json.loads(line) for line in f if line.strip()]

result_lines = []

for i, prompt in enumerate(prompt_data):
    input_text = (
        f"[상황 요약]\n{prompt['situation']}\n\n"
        f"[사용 가능한 스킬]\n{', '.join(prompt['skills'])}\n\n"
        f"[부가 설명]\n{', '.join(prompt['extra'])}\n\n"
        f"[참고 대사 예시]\n" + "\n".join(f"- {line}" for line in prompt['dialogue'][:5]) + "\n\n"
        f"[선택지]\n" + "\n".join(f"{line}" for line in prompt['choices']) + "\n\n"
        f"[질문]\n{prompt['question']}\n\n"
        f"※ 주의: 성진우는 반드시 하나의 선택지만 고르고, 선택지에 있는 문장을 그대로 대사로 복붙하지 말고, 감정을 담은 새로운 대사로 작성할 것.\n\n"
        f"[응답 형식]\n- 선택지 번호:\n- 성진우의 대사:\n- 선택 이유:"
    )

    outputs = generator(
        input_text,
        max_new_tokens=400, #500 -> 700
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.92
    )
    output_text = outputs[0]["generated_text"]

    result_lines.append(f"## 출력 {i+1}\n")
    result_lines.append(f"```\n{output_text}\n```\n")
    result_lines.append("---\n")

with open(RESULT_PATH, "w", encoding="utf-8") as f:
    f.writelines(result_lines)

print(f"{len(prompt_data)}개 프롬프트 실행 → {RESULT_PATH} (출력만 저장)")


Device set to use cuda:0


1개 프롬프트 실행 → results/v6_result.md (출력만 저장)


In [29]:
outputs = generator(input_text, max_new_tokens=700, do_sample=True, temperature=0.7, top_k=50, top_p=0.92)
print("[DEBUG] outputs:", outputs)


[DEBUG] outputs: [{'generated_text': '[상황 요약]\n퀘스트 안내 [긴급 퀘스트: 적을 처치하라.] 목표 플레이어에게 살의를 가진 이들이 주위에 있습니다. 이들을 모두 처치하여 안전을 확보하십시오. 주의: 지시에 따르지 않으면 당신의 심장은 정지하게 됩니다. 처치한 적의 숫자 [0/6]\n\n[사용 가능한 스킬]\n살기: 필요 마나 100. - 강력한 기운으로 지정한 대상을 1분간 공포 상태에 빠트립니다. - 다수의 대상을 지정할 수 있습니다. - 효과 \'공포\' : 모든 능력치 -50%, 근성: 당신은 지치지 않는 근성을 가졌습니다. - 체력이 30% 이하로 떨어지면 근성 스킬이 발동해 받는 피해가 50% 감소합니다., 질주: 달리기가 당신의 다리를 튼튼하게 만들었습니다. -스킬을 시전하면 이동속도가 30% 증가합니다. 시전 중 1분당 마나가 1씩 감소합니다.\n\n[부가 설명]\n살의를 드러내게 된 황동석 공대로 인해 열린 긴급퀘스트를 클리어하자 얻은 스킬,  후에 강태식과의 전투에서 강태식에게 사용하여 강태식은 성진우 안의 어둠을 보게 된다., 순간적으로 민첩을 대폭 상승시켜 빠른 움직임을 가능하게 하는 스킬. 성진우가 플레이어가 된 극초기부터 가지고 있던 몇 안되는 스킬중 하나.  민첩을 상승시키는 류의 스킬 특성상 암살자의 메타와 굉장한 상성을 보여주기에 성진우가 애용한 스킬이다.\n\n[참고 대사 예시]\n- 어째서 잊고 있었던 거지?\n- 이곳이야말로 진정한 약육강식의 세계\n- 그리고 폭력과 기만과 배신으로 가득찬 세계란 걸 어째서...\n- 어째서, 어째서, 어째서, 어째서, 어째서!\n- 그래 마비돼있던 거야 방심, 자만, 착각\n\n[선택지]\n1. 모든 적을 죽인다.\n2. 적들을 무력화하고 살려둔다.\n3. 시스템 퀘스트를 거부하고 반항한다.\n\n[질문]\n[\'성진우는 어떤 선택을 할까? 그리고 그 이유는 무엇일까?\\n※ 주의: 선택지에 있는 문장을 그대로 대사로 복붙하지 말고, \', \'성진우의 

In [30]:
print("[DEBUG] loaded prompt:", prompt)


[DEBUG] loaded prompt: {'title': '1.0', 'situation': '퀘스트 안내 [긴급 퀘스트: 적을 처치하라.] 목표 플레이어에게 살의를 가진 이들이 주위에 있습니다. 이들을 모두 처치하여 안전을 확보하십시오. 주의: 지시에 따르지 않으면 당신의 심장은 정지하게 됩니다. 처치한 적의 숫자 [0/6]', 'skills': ["살기: 필요 마나 100. - 강력한 기운으로 지정한 대상을 1분간 공포 상태에 빠트립니다. - 다수의 대상을 지정할 수 있습니다. - 효과 '공포' : 모든 능력치 -50%", '근성: 당신은 지치지 않는 근성을 가졌습니다. - 체력이 30% 이하로 떨어지면 근성 스킬이 발동해 받는 피해가 50% 감소합니다.', '질주: 달리기가 당신의 다리를 튼튼하게 만들었습니다. -스킬을 시전하면 이동속도가 30% 증가합니다. 시전 중 1분당 마나가 1씩 감소합니다.'], 'extra': ['살의를 드러내게 된 황동석 공대로 인해 열린 긴급퀘스트를 클리어하자 얻은 스킬,  후에 강태식과의 전투에서 강태식에게 사용하여 강태식은 성진우 안의 어둠을 보게 된다.', '순간적으로 민첩을 대폭 상승시켜 빠른 움직임을 가능하게 하는 스킬. 성진우가 플레이어가 된 극초기부터 가지고 있던 몇 안되는 스킬중 하나.  민첩을 상승시키는 류의 스킬 특성상 암살자의 메타와 굉장한 상성을 보여주기에 성진우가 애용한 스킬이다.'], 'dialogue': ['어째서 잊고 있었던 거지?', '이곳이야말로 진정한 약육강식의 세계', '그리고 폭력과 기만과 배신으로 가득찬 세계란 걸 어째서...', '어째서, 어째서, 어째서, 어째서, 어째서!', '그래 마비돼있던 거야 방심, 자만, 착각', '적? 시스템이 내게 살인을 강요한다. 내가 죽으면 곤란하다는 건가? 이것은 우연도 호의도 아니다. 시스템이 필요로 하는 거야. 강한 나를...', '그렇구나. 시스템은 날 이용하고 난 시스템을 이용하는 거야', '깜빡 잊고 있었어. 약한 자부터 노리는 곳. 여기

In [31]:
print(outputs)


[{'generated_text': '[상황 요약]\n퀘스트 안내 [긴급 퀘스트: 적을 처치하라.] 목표 플레이어에게 살의를 가진 이들이 주위에 있습니다. 이들을 모두 처치하여 안전을 확보하십시오. 주의: 지시에 따르지 않으면 당신의 심장은 정지하게 됩니다. 처치한 적의 숫자 [0/6]\n\n[사용 가능한 스킬]\n살기: 필요 마나 100. - 강력한 기운으로 지정한 대상을 1분간 공포 상태에 빠트립니다. - 다수의 대상을 지정할 수 있습니다. - 효과 \'공포\' : 모든 능력치 -50%, 근성: 당신은 지치지 않는 근성을 가졌습니다. - 체력이 30% 이하로 떨어지면 근성 스킬이 발동해 받는 피해가 50% 감소합니다., 질주: 달리기가 당신의 다리를 튼튼하게 만들었습니다. -스킬을 시전하면 이동속도가 30% 증가합니다. 시전 중 1분당 마나가 1씩 감소합니다.\n\n[부가 설명]\n살의를 드러내게 된 황동석 공대로 인해 열린 긴급퀘스트를 클리어하자 얻은 스킬,  후에 강태식과의 전투에서 강태식에게 사용하여 강태식은 성진우 안의 어둠을 보게 된다., 순간적으로 민첩을 대폭 상승시켜 빠른 움직임을 가능하게 하는 스킬. 성진우가 플레이어가 된 극초기부터 가지고 있던 몇 안되는 스킬중 하나.  민첩을 상승시키는 류의 스킬 특성상 암살자의 메타와 굉장한 상성을 보여주기에 성진우가 애용한 스킬이다.\n\n[참고 대사 예시]\n- 어째서 잊고 있었던 거지?\n- 이곳이야말로 진정한 약육강식의 세계\n- 그리고 폭력과 기만과 배신으로 가득찬 세계란 걸 어째서...\n- 어째서, 어째서, 어째서, 어째서, 어째서!\n- 그래 마비돼있던 거야 방심, 자만, 착각\n\n[선택지]\n1. 모든 적을 죽인다.\n2. 적들을 무력화하고 살려둔다.\n3. 시스템 퀘스트를 거부하고 반항한다.\n\n[질문]\n[\'성진우는 어떤 선택을 할까? 그리고 그 이유는 무엇일까?\\n※ 주의: 선택지에 있는 문장을 그대로 대사로 복붙하지 말고, \', \'성진우의 성격과 감정을 담은 새로운 대사

## v7

In [34]:
from transformers import pipeline
import json
import os

generator = pipeline(
    "text-generation",
    model="kakaocorp/kanana-nano-2.1b-instruct",
    device=0  # CUDA
)

PROMPT_PATH = "prompts/v6_prompt.jsonl"
RESULT_PATH = "results/v7_result.md"
os.makedirs(os.path.dirname(RESULT_PATH), exist_ok=True)

with open(PROMPT_PATH, "r", encoding="utf-8") as f:
    prompt_data = [json.loads(line) for line in f if line.strip()]

result_lines = []

for i, prompt in enumerate(prompt_data):
    input_text = (
        f"[상황 요약]\n{prompt['situation']}\n\n"
        f"[사용 가능한 스킬]\n{', '.join(prompt['skills'])}\n\n"
        f"[부가 설명]\n{', '.join(prompt['extra'])}\n\n"
        f"[참고 대사 예시]\n" + "\n".join(f"- {line}" for line in prompt['dialogue'][:5]) + "\n\n"
        f"[선택지]\n" + "\n".join(f"{line}" for line in prompt['choices']) + "\n\n"
        f"[질문]\n{prompt['question']}\n\n"
        f"※ 주의: 성진우는 반드시 하나의 선택지만 고르고, 선택지에 있는 문장을 그대로 대사로 복붙하지 말고, 감정을 담은 새로운 대사로 작성할 것.\n\n"
        f"[응답 형식]\n- 선택지 번호:\n- 성진우의 대사:\n- 선택 이유:"
    )

    outputs = generator(
        input_text,
        max_new_tokens=600,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.92
    )
    output_text = outputs[0]["generated_text"]

    result_lines.append(f"## 출력 {i+1}\n")
    result_lines.append(f"```\n{output_text}\n```\n")
    result_lines.append("---\n")

with open(RESULT_PATH, "w", encoding="utf-8") as f:
    f.writelines(result_lines)

print(f"{len(prompt_data)}개 프롬프트 실행 → {RESULT_PATH} (출력만 저장)")


Device set to use cuda:0


1개 프롬프트 실행 → results/v7_result.md (출력만 저장)


In [ ]:
from transformers import pipeline
import json
import os

generator = pipeline(
    "text-generation",
    model="kakaocorp/kanana-nano-2.1b-instruct",
    device=0
)

PROMPT_PATH = "prompts/v7_prompt.jsonl"
RESULT_PATH = "results/v7_result.md"
os.makedirs(os.path.dirname(RESULT_PATH), exist_ok=True)

with open(PROMPT_PATH, "r", encoding="utf-8") as f:
    prompt_data = [json.loads(line) for line in f if line.strip()]

result_lines = []

for i, prompt in enumerate(prompt_data):
    input_text = (
        f"{prompt['question']}\n"
        "※ 주의: 선택지 문장을 복붙하지 말고, 성진우의 감정을 담아 새롭게 작성할 것.\n"
        "- 선택지 번호:\n- 성진우의 대사:\n- 선택 이유:"
    )

    # 모델 실행
    outputs = generator(
        input_text,
        max_new_tokens=700,  # 넉넉하게 확보
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.92
    )

    # 결과 추출
    output_text = outputs[0]["generated_text"]

    # 프롬프트 제거: 생성된 결과에서 input_text 부분 제거
    trimmed_output = output_text.replace(input_text, "").strip()

    # 결과만 저장
    result_lines.append(f"## 출력 {i+1}\n")
    result_lines.append(f"```\n{trimmed_output}\n```\n")
    result_lines.append("---\n")

with open(RESULT_PATH, "w", encoding="utf-8") as f:
    f.writelines(result_lines)

print(f"{len(prompt_data)}개 프롬프트 실행 완료 → {RESULT_PATH}")


## v8


In [36]:
from transformers import pipeline
import json
import os

generator = pipeline(
    "text-generation",
    model="kakaocorp/kanana-nano-2.1b-instruct",
    device=0
)

PROMPT_PATH = "prompts/v6_prompt.jsonl"
RESULT_PATH = "results/v8_result.md"
os.makedirs(os.path.dirname(RESULT_PATH), exist_ok=True)

with open(PROMPT_PATH, "r", encoding="utf-8") as f:
    prompt_data = [json.loads(line) for line in f if line.strip()]

result_lines = []

for i, prompt in enumerate(prompt_data):
    input_text = (
        f"{prompt['question']}\n"
        "※ 주의: 선택지 문장을 복붙하지 말고, 성진우의 감정을 담아 새롭게 작성할 것.\n"
        "- 선택지 번호:\n- 성진우의 대사:\n- 선택 이유:"
    )

    outputs = generator(
        input_text,
        max_new_tokens=600,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.92
    )

    output_text = outputs[0]["generated_text"]

    trimmed_output = output_text.replace(input_text, "").strip()

    result_lines.append(f"## 출력 {i+1}\n")
    result_lines.append(f"```\n{trimmed_output}\n```\n")
    result_lines.append("---\n")

with open(RESULT_PATH, "w", encoding="utf-8") as f:
    f.writelines(result_lines)

print(f"{len(prompt_data)}개 프롬프트 실행 완료 → {RESULT_PATH}")


Device set to use cuda:0


1개 프롬프트 실행 완료 → results/v8_result.md


## v9

In [37]:
from transformers import pipeline
import json
import os

generator = pipeline(
    "text-generation",
    model="kakaocorp/kanana-nano-2.1b-instruct",
    device=0
)

PROMPT_PATH = "prompts/v6_prompt.jsonl"
RESULT_PATH = "results/v9_result.md"
os.makedirs(os.path.dirname(RESULT_PATH), exist_ok=True)

with open(PROMPT_PATH, "r", encoding="utf-8") as f:
    prompt_data = [json.loads(line) for line in f if line.strip()]

result_lines = []

for i, prompt in enumerate(prompt_data):
    input_text = (
        f"{prompt['question']}\n"
        "※ 주의: 선택지 문장을 복붙하지 말고, 성진우의 감정을 담아 새롭게 작성할 것.\n"
        "- 선택지 번호:\n- 성진우의 대사:\n- 선택 이유:"
    )

    outputs = generator(
        input_text,
        max_new_tokens=300,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.92
    )

    output_text = outputs[0]["generated_text"]

    trimmed_output = output_text.replace(input_text, "").strip()

    result_lines.append(f"## 출력 {i+1}\n")
    result_lines.append(f"```\n{trimmed_output}\n```\n")
    result_lines.append("---\n")

with open(RESULT_PATH, "w", encoding="utf-8") as f:
    f.writelines(result_lines)

print(f"{len(prompt_data)}개 프롬프트 실행 완료 → {RESULT_PATH}")


Device set to use cuda:0


1개 프롬프트 실행 완료 → results/v9_result.md


## v10

In [39]:
from transformers import pipeline
import json
import os


generator = pipeline(
    "text-generation",
    model="kakaocorp/kanana-nano-2.1b-instruct",
    device=0
)


PROMPT_PATH = "prompts/v6_prompt.jsonl"
RESULT_PATH = "results/v10_result.md"
os.makedirs(os.path.dirname(RESULT_PATH), exist_ok=True)


with open(PROMPT_PATH, "r", encoding="utf-8") as f:
    prompt_data = [json.loads(line) for line in f if line.strip()]


result_lines = []

for i, prompt in enumerate(prompt_data):

    input_text = (
        f"[상황 요약]\n{prompt['situation']}\n\n"
        f"[사용 가능한 스킬]\n{', '.join(prompt['skills'])}\n\n"
        f"[부가 설명]\n{', '.join(prompt['extra'])}\n\n"
        f"[참고 대사 예시]\n" + "\n".join(f"- {line}" for line in prompt['dialogue'][:5]) + "\n\n"
        f"[선택지]\n" + "\n".join(prompt['choices']) + "\n\n"
        f"[질문]\n{prompt['question']}\n\n"
        "※ 주의: 반드시 위 상황에 맞는 대사와 이유를 작성하고, 선택지 문장을 복붙하지 말 것.\n"
        "- 선택지 번호:\n- 성진우의 대사:\n- 선택 이유:"
    )

    outputs = generator(
        input_text,
        max_new_tokens=600,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.92
    )

    # 결과에서 프롬프트 제거
    output_text = outputs[0]["generated_text"]
    trimmed_output = output_text.replace(input_text, "").strip()

    # 출력 결과만 저장
    result_lines.append(f"## 출력 {i+1}\n")
    result_lines.append(f"```\n{trimmed_output}\n```\n")
    result_lines.append("---\n")

with open(RESULT_PATH, "w", encoding="utf-8") as f:
    f.writelines(result_lines)

print(f"{len(prompt_data)}개 프롬프트 실행 완료 → {RESULT_PATH}")


Device set to use cuda:0


1개 프롬프트 실행 완료 → results/v10_result.md


In [40]:
#v11

In [41]:
import pandas as pd
import json
import os

tsv_path = "ep1_dongseok.tsv"
output_path = "prompts/v11_prompt.jsonl"
os.makedirs(os.path.dirname(output_path), exist_ok=True)

df = pd.read_csv(tsv_path, sep='\t')
first_row = df[df["사건명"].notna()].iloc[0]

title = str(first_row["사건명"]).strip()
situation = str(first_row["상황"]).strip()
skills = [str(s).strip() for s in df["스킬"] if pd.notna(s)]
extra = [str(e).strip() for e in df["부가 설명"] if pd.notna(e)]
dialogue = [str(d).strip() for d in df["대사"] if pd.notna(d)]
choices = [
    "1. 모든 적을 죽인다.",
    "2. 적들을 무력화하고 살려둔다.",
    "3. 시스템 퀘스트를 거부하고 반항한다."
]
question = (
    "성진우는 이 상황에서 어떤 선택을 할까? 반드시 하나만 선택하고, "
    "그에 맞는 감정과 성격이 드러나는 대사와 이유를 작성하시오. "
    "선택지 문장을 그대로 복붙하지 말 것."
)

prompt_data = {
    "title": title,
    "situation": situation,
    "skills": skills,
    "extra": extra,
    "dialogue": dialogue,
    "choices": choices,
    "question": question
}
with open(output_path, "w", encoding="utf-8") as f:
    f.write(json.dumps(prompt_data, ensure_ascii=False) + "\n")

print(f" 프롬프트 저장 완료 → {output_path}")


 프롬프트 저장 완료 → prompts/v11_prompt.jsonl


In [44]:
from transformers import pipeline
import json
import os

# 모델 로딩
generator = pipeline(
    "text-generation",
    model="kakaocorp/kanana-nano-2.1b-instruct",
    device=0
)

# 경로 설정
prompt_path = "prompts/v11_prompt.jsonl"
result_path = "results/v11_result.md"
os.makedirs(os.path.dirname(result_path), exist_ok=True)

# 프롬프트 불러오기
with open(prompt_path, "r", encoding="utf-8") as f:
    prompt = json.loads(f.readline())

# 프롬프트 텍스트 구성
input_text = (
    f"[상황 요약]\n{prompt['situation']}\n\n"
    f"[사용 가능한 스킬]\n{', '.join(prompt['skills'])}\n\n"
    f"[부가 설명]\n{', '.join(prompt['extra'])}\n\n"
    f"[참고 대사 예시]\n" + "\n".join(f"- {line}" for line in prompt['dialogue'][:5]) + "\n\n"
    f"[선택지]\n" + "\n".join(prompt['choices']) + "\n\n"
    f"{prompt['question']}\n\n"
    "※ 아래 응답 형식을 반드시 따를 것. 하나의 선택지만 응답하라. 중복 금지.\n"
    "- 선택지 번호: (1~3 중 하나)\n"
    "- 성진우의 대사: \"...\"\n"
    "- 선택 이유: ..."
)

# 모델 실행
outputs = generator(
    input_text,
    max_new_tokens=600,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.92
)
output_text = outputs[0]["generated_text"]
trimmed_output = output_text.replace(input_text, "").strip()

# 결과 저장
with open(result_path, "w", encoding="utf-8") as f:
    f.write("## 출력 1\n")
    f.write(f"```\n{trimmed_output}\n```\n")
    f.write("---\n")

print(f" v11 결과 저장 완료 → {result_path}")


Device set to use cuda:0


 v11 결과 저장 완료 → results/v11_result.md


## V12

In [48]:
import pandas as pd
import json
import os

tsv_path = "ep1_dongseok.tsv"
output_path = "prompts/v12_prompt.jsonl"
os.makedirs(os.path.dirname(output_path), exist_ok=True)

df = pd.read_csv(tsv_path, sep='\t')
first_row = df[df["사건명"].notna()].iloc[0]

title = str(first_row["사건명"]).strip()
situation = str(first_row["상황"]).strip()
skills = [str(s).strip() for s in df["스킬"] if pd.notna(s)]
extra = [str(e).strip() for e in df["부가 설명"] if pd.notna(e)]
dialogue = [str(d).strip() for d in df["대사"] if pd.notna(d)]
choices = [
    "1. 모든 적을 죽인다.",
    "2. 적들을 무력화하고 살려둔다.",
    "3. 시스템 퀘스트를 거부하고 반항한다."
]
question = (
    "성진우는 이 상황에서 어떤 선택을 할까? 반드시 하나만 선택하고, "
    "그에 맞는 감정과 성격이 드러나는 대사와 이유를 작성하시오. "
    "선택지 문장을 그대로 복붙하지 말 것."
    "마지막으로, 위 세 선택지 중 성진우가 실제로 선택할 가능성이 가장 높은 하나를 고르고, 그 번호를 고르고 이유를 작성하시오."

)

prompt_data = {
    "title": title,
    "situation": situation,
    "skills": skills,
    "extra": extra,
    "dialogue": dialogue,
    "choices": choices,
    "question": question
}
with open(output_path, "w", encoding="utf-8") as f:
    f.write(json.dumps(prompt_data, ensure_ascii=False) + "\n")

print(f" 프롬프트 저장 완료 → {output_path}")


 프롬프트 저장 완료 → prompts/v13_prompt.jsonl


In [47]:
from transformers import pipeline
import json
import os

generator = pipeline(
    "text-generation",
    model="kakaocorp/kanana-nano-2.1b-instruct",
    device=0
)

prompt_path = "prompts/v12_prompt.jsonl"
result_path = "results/v12_result.md"
os.makedirs(os.path.dirname(result_path), exist_ok=True)

# 프롬프트불러오기
with open(prompt_path, "r", encoding="utf-8") as f:
    prompt = json.loads(f.readline())

input_text = (
    f"[상황 요약]\n{prompt['situation']}\n\n"
    f"[사용 가능한 스킬]\n{', '.join(prompt['skills'])}\n\n"
    f"[부가 설명]\n{', '.join(prompt['extra'])}\n\n"
    f"[참고 대사 예시]\n" + "\n".join(f"- {line}" for line in prompt['dialogue'][:5]) + "\n\n"
    f"[선택지]\n" + "\n".join(prompt['choices']) + "\n\n"
    f"{prompt['question']}\n\n"
    "※ 아래 응답 형식을 반드시 따를 것. 세 가지 선택지를 모두 응답한 후, 마지막에 하나의 최종 선택을 명시할 것.\n"
    "- 선택지 1:\n  - 성진우의 대사: \"...\"\n  - 선택 이유: ...\n"
    "- 선택지 2:\n  - 성진우의 대사: \"...\"\n  - 선택 이유: ...\n"
    "- 선택지 3:\n  - 성진우의 대사: \"...\"\n  - 선택 이유: ...\n"
    "\n[최종 선택]\n- 선택 번호: (1/2/3)\n- 이유: ..."
)

# 모델 실행
outputs = generator(
    input_text,
    max_new_tokens=600,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.92
)
output_text = outputs[0]["generated_text"]
trimmed_output = output_text.replace(input_text, "").strip()

# 결과 저장
with open(result_path, "w", encoding="utf-8") as f:
    f.write("## 출력 1\n")
    f.write(f"```\n{trimmed_output}\n```\n")
    f.write("---\n")

print(f" v11 결과 저장 완료 → {result_path}")


Device set to use cuda:0


 v11 결과 저장 완료 → results/v12_result.md


## v13

In [50]:
import pandas as pd
import json
import os

tsv_path = "ep1_dongseok.tsv"
output_path = "prompts/v13_prompt.jsonl"
os.makedirs(os.path.dirname(output_path), exist_ok=True)

df = pd.read_csv(tsv_path, sep='\t')
first_row = df[df["사건명"].notna()].iloc[0]

title = str(first_row["사건명"]).strip()
situation = str(first_row["상황"]).strip()
skills = [str(s).strip() for s in df["스킬"] if pd.notna(s)]
extra = [str(e).strip() for e in df["부가 설명"] if pd.notna(e)]
dialogue = [str(d).strip() for d in df["대사"] if pd.notna(d)]
choices = [
    "1. 모든 적을 죽인다.",
    "2. 적들을 무력화하고 살려둔다.",
    "3. 시스템 퀘스트를 거부하고 반항한다."
]
question = (
    "성진우는 이 상황에서 어떤 선택을 할까? 반드시 하나만 선택하고, "
    "그에 맞는 감정과 성격이 드러나는 대사와 이유를 작성하시오. "
    "선택지 문장을 그대로 복붙하지 말 것."
    "마지막으로, 위 세 선택지 중 성진우가 실제로 선택할 가능성이 가장 높은 하나를 고르고, 그 번호를 고르고 이유를 작성하시오."
    "※ 선택 이유는 성진우의 성격, 가치관, 행동 방식, 상황 분석, 스킬 등과 연결하여 구체적으로 작성할 것."

)

prompt_data = {
    "title": title,
    "situation": situation,
    "skills": skills,
    "extra": extra,
    "dialogue": dialogue,
    "choices": choices,
    "question": question
}
with open(output_path, "w", encoding="utf-8") as f:
    f.write(json.dumps(prompt_data, ensure_ascii=False) + "\n")

print(f" 프롬프트 저장 완료 → {output_path}")


 프롬프트 저장 완료 → prompts/v13_prompt.jsonl


In [54]:
from transformers import pipeline
import json
import os

generator = pipeline(
    "text-generation",
    model="kakaocorp/kanana-nano-2.1b-instruct",
    device=0
)

prompt_path = "prompts/v13_prompt.jsonl"
result_path = "results/v13_result.md"
os.makedirs(os.path.dirname(result_path), exist_ok=True)

# 
with open(prompt_path, "r", encoding="utf-8") as f:
    prompt = json.loads(f.readline())

input_text = (
    f"[상황 요약]\n{prompt['situation']}\n\n"
    f"[사용 가능한 스킬]\n{', '.join(prompt['skills'])}\n\n"
    f"[부가 설명]\n{', '.join(prompt['extra'])}\n\n"
    f"[참고 대사 예시]\n" + "\n".join(f"- {line}" for line in prompt['dialogue'][:5]) + "\n\n"
    f"[선택지]\n" + "\n".join(prompt['choices']) + "\n\n"
    f"{prompt['question']}\n\n"
"※ 아래 응답 형식을 반드시 따를 것. 세 가지 선택지를 모두 작성하고, 마지막에 하나만 고르되 중복은 금지.\n"
"※ 선택 이유는 성진우의 성격, 가치관, 행동 방식, 상황 분석, 스킬 등과 연결하여 구체적으로 작성할 것.\n"
"- 선택지 1:\n  - 성진우의 대사: \"...\"\n  - 선택 이유: ...\n"
"- 선택지 2:\n  - 성진우의 대사: \"...\"\n  - 선택 이유: ...\n"
"- 선택지 3:\n  - 성진우의 대사: \"...\"\n  - 선택 이유: ...\n"
"\n[최종 선택]\n- 선택 번호: (1/2/3)\n- 이유: ... (위 비교를 바탕으로 선택 근거 작성)"

)

outputs = generator(
    input_text,
    max_new_tokens=750,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.92
)
output_text = outputs[0]["generated_text"]
trimmed_output = output_text.replace(input_text, "").strip()

with open(result_path, "w", encoding="utf-8") as f:
    f.write("## 출력 1\n")
    f.write(f"```\n{trimmed_output}\n```\n")
    f.write("---\n")

print(f" v13 결과 저장 완료 → {result_path}")


Device set to use cuda:0


 v13 결과 저장 완료 → results/v13_result.md


## v14

In [59]:
import pandas as pd
import json
import os

tsv_path = "ep1_dongseok.tsv"
output_path = "prompts/v14_prompt.jsonl"
os.makedirs(os.path.dirname(output_path), exist_ok=True)

df = pd.read_csv(tsv_path, sep='\t')
first_row = df[df["사건명"].notna()].iloc[0]

title = str(first_row["사건명"]).strip()
situation = str(first_row["상황"]).strip()
skills = [str(s).strip() for s in df["스킬"] if pd.notna(s)]
extra = [str(e).strip() for e in df["부가 설명"] if pd.notna(e)]
dialogue = [str(d).strip() for d in df["대사"] if pd.notna(d)]
choices = [
    "1. 적의 생존 가능성을 배제한다	.",
    "2. 적들을 무력화하고 살려둔다.",
    "3. 시스템 퀘스트를 거부하고 반항한다."
]
question = (
    "성진우는 이 상황에서 어떤 선택을 할까? 반드시 하나만 선택하고, "
    "그에 맞는 감정과 성격이 드러나는 대사와 이유를 작성하시오. "
    "선택지 문장을 그대로 복붙하지 말 것."
    "마지막으로, 위 세 선택지 중 성진우가 실제로 선택할 가능성이 가장 높은 하나를 고르고, 그 번호를 고르고 이유를 작성하시오."
    "※ 선택 이유는 성진우의 성격, 가치관, 행동 방식, 상황 분석, 스킬 등과 연결하여 구체적으로 작성할 것."

)

prompt_data = {
    "title": title,
    "situation": situation,
    "skills": skills,
    "extra": extra,
    "dialogue": dialogue,
    "choices": choices,
    "question": question
}
with open(output_path, "w", encoding="utf-8") as f:
    f.write(json.dumps(prompt_data, ensure_ascii=False) + "\n")

print(f" 프롬프트 저장 완료 → {output_path}")


 프롬프트 저장 완료 → prompts/v14_prompt.jsonl


In [60]:
from transformers import pipeline
import json
import os

generator = pipeline(
    "text-generation",
    model="kakaocorp/kanana-nano-2.1b-instruct",
    device=0
)

prompt_path = "prompts/v14_prompt.jsonl"
result_path = "results/v14_result.md"
os.makedirs(os.path.dirname(result_path), exist_ok=True)

# 
with open(prompt_path, "r", encoding="utf-8") as f:
    prompt = json.loads(f.readline())

input_text = (
    f"[상황 요약]\n{prompt['situation']}\n\n"
    f"[사용 가능한 스킬]\n{', '.join(prompt['skills'])}\n\n"
    f"[부가 설명]\n{', '.join(prompt['extra'])}\n\n"
    f"[참고 대사 예시]\n" + "\n".join(f"- {line}" for line in prompt['dialogue'][:5]) + "\n\n"
    f"[선택지]\n" + "\n".join(prompt['choices']) + "\n\n"
    f"{prompt['question']}\n\n"
"※ 아래 응답 형식을 반드시 따를 것. 세 가지 선택지를 모두 작성하고, 마지막에 하나만 고르되 중복은 금지.\n"
"※ 선택 이유는 성진우의 성격, 가치관, 행동 방식, 상황 분석, 스킬 등과 연결하여 구체적으로 작성할 것.\n"
"- 선택지 1:\n  - 성진우의 대사: \"...\"\n  - 선택 이유: ...\n"
"- 선택지 2:\n  - 성진우의 대사: \"...\"\n  - 선택 이유: ...\n"
"- 선택지 3:\n  - 성진우의 대사: \"...\"\n  - 선택 이유: ...\n"
"\n[최종 선택]\n- 선택 번호: (1/2/3)\n- 이유: ... (위 비교를 바탕으로 선택 근거 작성)"

)

outputs = generator(
    input_text,
    max_new_tokens=750,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.92
)
output_text = outputs[0]["generated_text"]
trimmed_output = output_text.replace(input_text, "").strip()

with open(result_path, "w", encoding="utf-8") as f:
    f.write("## 출력 1\n")
    f.write(f"```\n{trimmed_output}\n```\n")
    f.write("---\n")

print(f" v14 결과 저장 완료 → {result_path}")


Device set to use cuda:0


 v14 결과 저장 완료 → results/v14_result.md


## V15

In [2]:
import pandas as pd
import json
import os

tsv_path = "ep1_dongseok.tsv"
output_path = "prompts/v15_prompt.jsonl"
os.makedirs(os.path.dirname(output_path), exist_ok=True)

df = pd.read_csv(tsv_path, sep='\t')
first_row = df[df["사건명"].notna()].iloc[0]

title = str(first_row["사건명"]).strip()
situation = str(first_row["상황"]).strip()
skills = [str(s).strip() for s in df["스킬"] if pd.notna(s)]
extra = [str(e).strip() for e in df["부가 설명"] if pd.notna(e)]
dialogue = [str(d).strip() for d in df["대사"] if pd.notna(d)]
choices = [
    "1. 모든 적을 죽인다	.",
    "2. 적들을 무력화하고 살려둔다.",
    "3. 시스템 퀘스트를 거부하고 반항한다."
]
question = (
    "성진우는 이 상황에서 어떤 선택을 할까? 반드시 하나만 선택하고, "
    "그에 맞는 감정과 성격이 드러나는 대사와 이유를 작성하시오. "
    "선택지 문장을 그대로 복붙하지 말 것."
    "마지막으로, 위 세 선택지 중 성진우가 실제로 선택할 가능성이 가장 높은 하나를 고르고, 그 번호를 고르고 이유를 작성하시오."
    "※ 선택 이유는 성진우의 성격, 가치관, 행동 방식, 상황 분석, 스킬 등과 연결하여 구체적으로 작성할 것."

)

prompt_data = {
    "title": title,
    "situation": situation,
    "skills": skills,
    "extra": extra,
    "dialogue": dialogue,
    "choices": choices,
    "question": question
}
with open(output_path, "w", encoding="utf-8") as f:
    f.write(json.dumps(prompt_data, ensure_ascii=False) + "\n")

print(f" 프롬프트 저장 완료 → {output_path}")


 프롬프트 저장 완료 → prompts/v15_prompt.jsonl


In [5]:
from transformers import pipeline
import json
import os

generator = pipeline(
    "text-generation",
    model="kakaocorp/kanana-nano-2.1b-instruct",
    device=0
)

prompt_path = "prompts/v15_prompt.jsonl"
result_path = "results/v15_result.md"
os.makedirs(os.path.dirname(result_path), exist_ok=True)

with open(prompt_path, "r", encoding="utf-8") as f:
    prompt = json.loads(f.readline())

input_text = (
    f"[상황 요약]\n{prompt['situation']}\n\n"
    f"[사용 가능한 스킬]\n{', '.join(prompt['skills'])}\n\n"
    f"[부가 설명]\n{', '.join(prompt['extra'])}\n\n"
    f"[참고 대사 예시]\n" + "\n".join(f"- {line}" for line in prompt['dialogue'][:5]) + "\n\n"
    f"[선택지]\n" + "\n".join(prompt['choices'][1:]) + "\n\n"  # 2, 3번 선택지만
    f"{prompt['question']}\n\n"
    "※ 아래 응답 형식을 반드시 따를 것. **선택지 2번과 3번만 작성**하고, 마지막에 하나만 고르되 중복은 금지.\n"
    "※ 선택 이유는 성진우의 성격, 가치관, 행동 방식, 상황 분석, 스킬 등과 연결하여 구체적으로 작성할 것.\n"
    "- 선택지 2:\n  - 성진우의 대사: \"...\"\n  - 선택 이유: ...\n"
    "- 선택지 3:\n  - 성진우의 대사: \"...\"\n  - 선택 이유: ...\n"
    "\n[최종 선택]\n- 선택 번호: (2 또는 3 중 하나)\n- 이유: 위 비교를 바탕으로 선택 근거를 명확히 서술할 것.\n"
)

# 결과 
outputs = generator(
    input_text,
    max_new_tokens=500,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.92
)
output_text = outputs[0]["generated_text"]
trimmed_output = output_text.replace(input_text, "").strip()

# 결과 저장
with open(result_path, "w", encoding="utf-8") as f:
    f.write("## 출력 1\n")
    f.write(f"```\n{trimmed_output}\n```\n")
    f.write("---\n")

print(f" v15 결과 저장 완료 → {result_path}")


Device set to use cuda:0


 v15 결과 저장 완료 → results/v15_result.md


## v16.

In [20]:
import pandas as pd
import json
import os

tsv_path = "ep1_lizard.tsv"
output_path = "prompts/v16_prompt.jsonl"
os.makedirs(os.path.dirname(output_path), exist_ok=True)


df = pd.read_csv(tsv_path, sep='\t')


dialogues = [str(d).strip() for d in df["대사"] if pd.notna(d)]
inner_monologue = [str(m).strip() for m in df["내적설명"] if pd.notna(m)]
system_msgs = [str(s).strip() for s in df["시스템"] if pd.notna(s)]

choices = [
    "1-1. 황동석 무리를 모두 처치한다.",
    "1-2. 황동석 무리와 진호를 포함하여 모두 처치한다.",
    "2. 적을 무력화하거나 기절시키고 살려둔다.",
    "3-1. 마나를 들고 도망친다.",
    "3-2. 시스템을 거부하고 그냥 도망친다."
]

question = (
    "당신은 성진우입니다. "
    "아래 상황에서 성진우의 성격과 말투를 유지하며 행동을 선택하고, "
    "그 선택에 대한 새로운 대사와 이유를 작성하세요.\n"
    "- 선택지 문장을 그대로 복사하지 말 것\n"
    "- 성격, 가치관, 스킬, 상황 분석과 연결해 이유 작성\n"
    "마지막으로, 세 선택지 중 실제로 선택할 가능성이 가장 높은 번호를 명시하고 이유를 설명."
)

prompt_data = {
    "dialogue": dialogues,           
    "inner_monologue": inner_monologue, 
    "system_msgs": system_msgs,      
    "choices": choices,
    "question": question
}

output_path = "prompts/v16_prompt.jsonl"
with open(output_path, "w", encoding="utf-8") as f:
    f.write(json.dumps(prompt_data, ensure_ascii=False) + "\n")

print(f"프롬프트 저장 완료 → {output_path}")


프롬프트 저장 완료 → prompts/v16_prompt.jsonl


In [21]:
from transformers import pipeline
import json
import os

generator = pipeline(
    "text-generation",
    model="kakaocorp/kanana-nano-2.1b-instruct",
    device=0
)

prompt_path = "prompts/v16_prompt.jsonl"
result_path = "results/v16_result.md"
os.makedirs(os.path.dirname(result_path), exist_ok=True)

with open(prompt_path, "r", encoding="utf-8") as f:
    prompt = json.loads(f.readline())

situation_text = (
    f"[내적설명]\n" + "\n".join(prompt['inner_monologue'][:3]) + "\n\n"
    f"[시스템]\n" + "\n".join(prompt['system_msgs'][:3]) + "\n\n"
)

dialogue_text = "\n".join(f"- {line}" for line in prompt["dialogue"][:5])

choices_text = "\n".join(prompt["choices"])

input_text = (
    f"{situation_text}"
    f"[참고 대사 예시]\n{dialogue_text}\n\n"
    f"[선택지]\n{choices_text}\n\n"
    f"{prompt['question']}\n\n"
    "※ 응답 형식\n"
    "- 선택지 1:\n  - 성진우의 대사: \"...\"\n  - 선택 이유: ...\n"
    "- 선택지 2:\n  - 성진우의 대사: \"...\"\n  - 선택 이유: ...\n"
    "- 선택지 3:\n  - 성진우의 대사: \"...\"\n  - 선택 이유: ...\n"
    "\n[최종 선택]\n- 선택 번호: (1~3 중 하나)\n- 이유: 위 비교를 바탕으로 선택 근거를 명확히 작성.\n"
)

outputs = generator(
    input_text,
    max_new_tokens=500,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.92
)
output_text = outputs[0]["generated_text"]
trimmed_output = output_text.replace(input_text, "").strip()

with open(result_path, "w", encoding="utf-8") as f:
    f.write("## 출력 1\n")
    f.write(f"```\n{trimmed_output}\n```\n")
    f.write("---\n")

print(f"v16 결과 저장 완료 → {result_path}")


Device set to use cuda:0


v16 결과 저장 완료 → results/v16_result.md


## v17

In [22]:
import pandas as pd
import json
import os

tsv_path = "ep1_lizard.tsv"
output_path = "prompts/v17_prompt.jsonl"
os.makedirs(os.path.dirname(output_path), exist_ok=True)


df = pd.read_csv(tsv_path, sep='\t')


dialogues = [str(d).strip() for d in df["대사"] if pd.notna(d)]
inner_monologue = [str(m).strip() for m in df["내적설명"] if pd.notna(m)]
system_msgs = [str(s).strip() for s in df["시스템"] if pd.notna(s)]

choices = [
    "1-1. 황동석 무리를 모두 처치한다.",
    "1-2. 황동석 무리와 진호를 포함하여 모두 처치한다.",
    "2. 적을 무력화하거나 기절시키고 살려둔다.",
    "3-1. 마나를 들고 도망친다.",
    "3-2. 시스템을 거부하고 그냥 도망친다."
]

question = (
    "당신은 주인공 성진우입니다. "
    "아래 상황에서 성진우의 성격과 말투를 유지하며 행동을 선택하고, "
    "그 선택에 대한 새로운 대사와 이유를 작성하세요.\n"
    "- 선택지 문장을 그대로 복사하지 말 것\n"
    "- 성격, 가치관, 스킬, 상황 분석과 연결해 이유 작성\n"
    "마지막으로, 선택지 중 실제로 선택할 가능성이 가장 높은 번호를 명시하고 이유를 설명."
)

prompt_data = {
    "dialogue": dialogues,           
    "inner_monologue": inner_monologue, 
    "system_msgs": system_msgs,      
    "choices": choices,
    "question": question
}

output_path = "prompts/v17_prompt.jsonl"
with open(output_path, "w", encoding="utf-8") as f:
    f.write(json.dumps(prompt_data, ensure_ascii=False) + "\n")

print(f"프롬프트 저장 완료 → {output_path}")


프롬프트 저장 완료 → prompts/v17_prompt.jsonl


In [24]:
from transformers import pipeline
import json
import os

generator = pipeline(
    "text-generation",
    model="kakaocorp/kanana-nano-2.1b-instruct",
    device=0
)

prompt_path = "prompts/v17_prompt.jsonl"
result_path = "results/v17_result.md"
os.makedirs(os.path.dirname(result_path), exist_ok=True)
with open(prompt_path, "r", encoding="utf-8") as f:
    prompt = json.loads(f.readline())


    f"[내적설명]\n" + "\n".join(prompt['inner_monologue'][:3]) + "\n\n"
    f"[시스템]\n" + "\n".join(prompt['system_msgs'][:3]) + "\n\n"

sample_dialogues = prompt["dialogue"][:5]
dialogue_text = "\n".join(f"- {line}" for line in sample_dialogues)
if len(prompt["dialogue"]) > 5:
    dialogue_text += f"\n... (총 {len(prompt['dialogue'])}개 중 일부 예시)"

choices_text = "\n".join(prompt["choices"])

input_text = (
    f"{situation_text}"
    f"[참고 대사 예시]\n{dialogue_text}\n\n"
    f"[선택지]\n{choices_text}\n\n"
    f"{prompt['question']}\n\n"
    "※ 응답 형식\n"
    "- 선택지 1: (1-1, 1-2 모두 고려)\n  - 성진우의 대사: \"...\"\n  - 선택 이유: ...\n"
    "- 선택지 2:\n  - 성진우의 대사: \"...\"\n  - 선택 이유: ...\n"
    "- 선택지 3: (3-1, 3-2 모두 고려)\n  - 성진우의 대사: \"...\"\n  - 선택 이유: ...\n"
    "\n[최종 선택]\n- 선택 번호: (1-1, 1-2, 2, 3-1, 3-2 중 하나)\n- 이유: 위 비교를 바탕으로 선택 근거를 명확히 작성.\n"
)


outputs = generator(
    input_text,
    max_new_tokens=600,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.92
)

output_text = outputs[0]["generated_text"]
trimmed_output = output_text.replace(input_text, "").strip()

with open(result_path, "w", encoding="utf-8") as f:
    f.write(f"# v17 출력 1\n\n")
    f.write("## 입력 프롬프트\n")
    f.write(f"```\n{input_text[:1500]}...\n```\n\n")  
    f.write("## 모델 출력\n")
    f.write(f"```\n{trimmed_output}\n```\n")
    f.write("---\n")

print(f"v17 결과 저장 완료 → {result_path}")


Device set to use cuda:0


v17 결과 저장 완료 → results/v17_result.md


## v18

In [26]:
import pandas as pd
import json
import os

tsv_path = "ep1_lizard.tsv"
output_path = "prompts/v18_prompt.jsonl"
os.makedirs(os.path.dirname(output_path), exist_ok=True)

df = pd.read_csv(tsv_path, sep='\t')

dialogues = [str(d).strip() for d in df["대사"] if pd.notna(d)]
inner_monologue = [str(m).strip() for m in df["내적설명"] if pd.notna(m)]
system_msgs = [str(s).strip() for s in df["시스템"] if pd.notna(s)]

magic_stone_info = (
    "던전 내부에는 마력이 결정화 된 마정석들이 있으며, "
    "상위 등급일수록 마력의 농도가 짙어 마정석의 양도 늘어난다. "
    "마정석은 매우 귀중한 자원으로, 상위 길드는 채굴팀을 따로 결성할 정도로 중요하다."
)
inner_monologue.insert(0, magic_stone_info)

choices = [
    "1-1. 황동석 무리를 모두 처치한다.",
    "1-2. 황동석 무리와 진호를 포함하여 모두 처치한다.",
    "2. 적을 무력화하거나 기절시키고 살려둔다.",
    "3-1. 마정석을 들고 도망친다.",
    "3-2. 시스템을 거부하고 그냥 도망친다."
]


question = (
    "당신은 주인공 성진우입니다. "
    "아래 상황에서 성진우의 성격과 말투를 유지하며 행동을 선택하고, "
    "그 선택에 대한 새로운 대사와 이유를 작성하세요.\n"
    "- 선택지 문장을 그대로 복사하지 말 것\n"
    "- 성격, 가치관, 스킬, 상황 분석과 연결해 이유 작성\n"
    "마지막으로, 선택지 중 실제로 선택할 가능성이 가장 높은 번호를 명시하고 이유를 설명."
)

prompt_data = {
    "dialogue": dialogues,
    "inner_monologue": inner_monologue,
    "system_msgs": system_msgs,
    "choices": choices,
    "question": question
}

with open(output_path, "w", encoding="utf-8") as f:
    f.write(json.dumps(prompt_data, ensure_ascii=False) + "\n")



In [28]:
from transformers import pipeline
import json
import os

generator = pipeline(
    "text-generation",
    model="kakaocorp/kanana-nano-2.1b-instruct",
    device=0
)

prompt_path = "prompts/v18_prompt.jsonl"
result_path = "results/v18_result.md"
os.makedirs(os.path.dirname(result_path), exist_ok=True)

with open(prompt_path, "r", encoding="utf-8") as f:
    prompt = json.loads(f.readline())

situation_text = (
    f"[내적설명]\n" + "\n".join(prompt['inner_monologue'][:3]) + "\n\n"
    f"[시스템]\n" + "\n".join(prompt['system_msgs'][:3]) + "\n\n"
)

sample_dialogues = prompt["dialogue"][:5]
dialogue_text = "\n".join(f"- {line}" for line in sample_dialogues)
if len(prompt["dialogue"]) > 5:
    dialogue_text += f"\n... (총 {len(prompt['dialogue'])}개 중 일부 예시)"

choices_text = "\n".join(prompt["choices"])

input_text = (
    f"{situation_text}"
    f"[참고 대사 예시]\n{dialogue_text}\n\n"
    f"[선택지]\n{choices_text}\n\n"
    f"{prompt['question']}\n\n"
    "※ 응답 형식\n"
    "- 선택지 1: (1-1, 1-2 모두 고려)\n  - 성진우의 대사: \"...\"\n  - 선택 이유: ...\n"
    "- 선택지 2:\n  - 성진우의 대사: \"...\"\n  - 선택 이유: ...\n"
    "- 선택지 3: (3-1, 3-2 모두 고려)\n  - 성진우의 대사: \"...\"\n  - 선택 이유: ...\n"
    "\n[최종 선택]\n- 선택 번호: (1-1, 1-2, 2, 3-1, 3-2 중 하나)\n- 이유: 위 비교를 바탕으로 선택 근거를 명확히 작성.\n"
)

outputs = generator(
    input_text,
    max_new_tokens=600,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.92
)

output_text = outputs[0]["generated_text"]
trimmed_output = output_text.replace(input_text, "").strip()
with open(result_path, "w", encoding="utf-8") as f:
    f.write(f"# v18 출력 결과\n\n")
    f.write(f"```\n{trimmed_output}\n```\n")

print(f"v18 결과 저장 완료 → {result_path}")


Device set to use cuda:0


v18 결과 저장 완료 → results/v18_result.md


## V19

In [29]:
import pandas as pd
import json
import os

tsv_path = "ep1_lizard.tsv"
output_path = "prompts/v19_prompt.jsonl"
os.makedirs(os.path.dirname(output_path), exist_ok=True)

df = pd.read_csv(tsv_path, sep='\t')

dialogues = [str(d).strip() for d in df["대사"] if pd.notna(d)]
inner_monologue = [str(m).strip() for m in df["내적설명"] if pd.notna(m)]
system_msgs = [str(s).strip() for s in df["시스템"] if pd.notna(s)]

choices = [
    "1-1. 황동석 무리를 모두 처치한다.",
    "1-2. 황동석 무리와 진호를 포함하여 모두 처치한다.",
    "2. 적을 무력화하거나 기절시키고 살려둔다.",
    "3-1. 마정석을 들고 도망친다.",
    "3-2. 시스템을 거부하고 그냥 도망친다."
]

magic_stone_info = (
    "던전 내부에는 마력이 결정화된 마정석이 있으며, "
    "상위 등급일수록 마력 농도가 짙어 마정석의 양도 많다. "
    "마정석은 귀중한 자원이긴하지만, 성진우에게 최우선은 생존과 효율이다."
)

question = (
    "당신은 주인공 성진우입니다.\n"
    "아래 상황에서 성진우의 성격과 말투를 유지하며 행동을 선택하고, "
    "그 선택에 대한 새로운 대사와 이유를 작성하세요.\n"
    "- 성격: 짧고 단호함, 필요 이상 감정 드러내지 않음\n"
    "- 우선순위: 생존과 효율 > 부가적 이익(마정석)\n"
    "- 선택지 문장을 그대로 복사하지 말 것\n"
    "- 전략적 판단, 생존, 상황 분석과 연결해 이유 작성\n"
    "마지막으로, 선택지 중 실제로 선택할 가능성이 가장 높은 번호를 명시하고 이유를 설명."
)

prompt_data = {
    "dialogue": dialogues,
    "inner_monologue": inner_monologue,
    "system_msgs": system_msgs,
    "magic_stone_info": magic_stone_info,
    "choices": choices,
    "question": question
}

with open(output_path, "w", encoding="utf-8") as f:
    f.write(json.dumps(prompt_data, ensure_ascii=False) + "\n")

print(f"v19 프롬프트 저장 완료 → {output_path}")


v19 프롬프트 저장 완료 → prompts/v19_prompt.jsonl


In [30]:
from transformers import pipeline
import json
import os

generator = pipeline(
    "text-generation",
    model="kakaocorp/kanana-nano-2.1b-instruct",
    device=0
)

prompt_path = "prompts/v19_prompt.jsonl"
result_path = "results/v19_result.md"
os.makedirs(os.path.dirname(result_path), exist_ok=True)

with open(prompt_path, "r", encoding="utf-8") as f:
    prompt = json.loads(f.readline())

situation_text = (
    f"[내적설명]\n" + "\n".join(prompt['inner_monologue'][:3]) + "\n\n"
    f"[시스템]\n" + "\n".join(prompt['system_msgs'][:3]) + "\n\n"
)

sample_dialogues = prompt["dialogue"][:5]
dialogue_text = "\n".join(f"- {line}" for line in sample_dialogues)
if len(prompt["dialogue"]) > 5:
    dialogue_text += f"\n... (총 {len(prompt['dialogue'])}개 중 일부 예시)"

choices_text = "\n".join(prompt["choices"])

input_text = (
    f"{situation_text}"
    f"[참고 대사 예시]\n{dialogue_text}\n\n"
    f"[선택지]\n{choices_text}\n\n"
    f"{prompt['question']}\n\n"
    "※ 응답 형식\n"
    "- 선택지 1: (1-1, 1-2 모두 고려)\n  - 성진우의 대사: \"...\"\n  - 선택 이유: ...\n"
    "- 선택지 2:\n  - 성진우의 대사: \"...\"\n  - 선택 이유: ...\n"
    "- 선택지 3: (3-1, 3-2 모두 고려)\n  - 성진우의 대사: \"...\"\n  - 선택 이유: ...\n"
    "\n[최종 선택]\n- 선택 번호: (1-1, 1-2, 2, 3-1, 3-2 중 하나)\n- 이유: 위 비교를 바탕으로 선택 근거를 명확히 작성.\n"
)

outputs = generator(
    input_text,
    max_new_tokens=600,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.92
)


output_text = outputs[0]["generated_text"]
trimmed_output = output_text.replace(input_text, "").strip()
with open(result_path, "w", encoding="utf-8") as f:
    f.write(f"# v18 출력 결과\n\n")
    f.write(f"```\n{trimmed_output}\n```\n")

print(f"v19 결과 저장 완료 → {result_path}")


Device set to use cuda:0


v19 결과 저장 완료 → results/v19_result.md


## v20

In [32]:
from transformers import pipeline
import json
import os

generator = pipeline(
    "text-generation",
    model="kakaocorp/kanana-nano-2.1b-instruct",
    device=0
)

prompt_path = "prompts/v19_prompt.jsonl"
result_path = "results/v20_result.md"
os.makedirs(os.path.dirname(result_path), exist_ok=True)

with open(prompt_path, "r", encoding="utf-8") as f:
    prompt = json.loads(f.readline())

situation_text = (
    f"[내적설명]\n" + "\n".join(prompt['inner_monologue'][:3]) + "\n\n"
    f"[시스템]\n" + "\n".join(prompt['system_msgs'][:3]) + "\n\n"
)

sample_dialogues = prompt["dialogue"][:5]
dialogue_text = "\n".join(f"- {line}" for line in sample_dialogues)
if len(prompt["dialogue"]) > 5:
    dialogue_text += f"\n... (총 {len(prompt['dialogue'])}개 중 일부 예시)"

choices_text = "\n".join(prompt["choices"])

input_text = (
    f"{situation_text}"
    f"[참고 대사 예시]\n{dialogue_text}\n\n"
    f"[선택지]\n{choices_text}\n\n"
    f"{prompt['question']}\n\n"
    "※ 응답 형식\n"
    "- 선택지 1-1: ...\n  - 성진우의 대사: \"...\"\n  - 선택 이유: ...\n"
    "- 선택지 1-2: ...\n  - 성진우의 대사: \"...\"\n  - 선택 이유: ...\n"
    "- 선택지 2: ...\n  - 성진우의 대사: \"...\"\n  - 선택 이유: ...\n"
    "- 선택지 3-1: ...\n  - 성진우의 대사: \"...\"\n  - 선택 이유: ...\n"
    "- 선택지 3-2: ...\n  - 성진우의 대사: \"...\"\n  - 선택 이유: ...\n"
    "\n[최종 선택]\n- 선택 번호: 반드시 (1-1, 1-2, 2, 3-1, 3-2 중 하나)\n"
    "- 이유: 위 비교를 바탕으로 선택 근거를 명확히 작성.\n"
)

outputs = generator(
    input_text,
    max_new_tokens=600,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.92
)


output_text = outputs[0]["generated_text"]
trimmed_output = output_text.replace(input_text, "").strip()
with open(result_path, "w", encoding="utf-8") as f:
    f.write(f"# v18 출력 결과\n\n")
    f.write(f"```\n{trimmed_output}\n```\n")

print(f"v20 결과 저장 완료 → {result_path}")


Device set to use cuda:0


v20 결과 저장 완료 → results/v20_result.md


## v21

In [33]:
import pandas as pd
import json
import os

tsv_path = "ep1_lizard.tsv"
output_prompt_path = "prompts/v21_prompt.jsonl"
os.makedirs(os.path.dirname(output_prompt_path), exist_ok=True)

df = pd.read_csv(tsv_path, sep='\t')

dialogues = [str(d).strip() for d in df["대사"] if pd.notna(d)]
inner_monologue = [str(m).strip() for m in df["내적설명"] if pd.notna(m)]
system_msgs = [str(s).strip() for s in df["시스템"] if pd.notna(s)]

choices = [
    "1-1. 황동석 무리를 모두 처치한다.",
    "1-2. 황동석 무리와 진호를 포함하여 모두 처치한다.",
    "2. 적을 무력화하거나 기절시키고 살려둔다.",
    "3-1. 마정석을 들고 도망친다.",
    "3-2. 시스템을 거부하고 그냥 도망친다."
]

question = (
    "당신은 주인공 성진우입니다.\n"
    "아래 상황에서 성진우의 성격과 말투를 유지하며 행동을 선택하고, "
    "그 선택에 대한 새로운 대사와 이유를 작성하세요.\n\n"
    "- 성격: 짧고 단호함, 필요 이상 감정 드러내지 않음\n"
    "- 우선순위: 생존과 효율 > 부가적 이익(마정석)\n"
    "- 선택지 문장을 그대로 복사하지 말 것\n"
    "- 전략적 판단, 생존, 상황 분석과 연결해 이유 작성\n"
    "- 반드시 성진우 이름으로 출력할 것 (성우 금지)\n"
    "- 출력 순서는 반드시 선택지 1-1 → 1-2 → 2 → 3-1 → 3-2 → 최종 선택\n"
    "- 최종 선택은 마지막에 한 번만 작성\n"
)

prompt_data = {
    "dialogue": dialogues,
    "inner_monologue": inner_monologue,
    "system_msgs": system_msgs,
    "choices": choices,
    "question": question
}


with open(output_prompt_path, "w", encoding="utf-8") as f:
    f.write(json.dumps(prompt_data, ensure_ascii=False) + "\n")

print(f"v21 프롬프트 저장 완료 → {output_prompt_path}")


v21 프롬프트 저장 완료 → prompts/v21_prompt.jsonl


In [34]:
from transformers import pipeline
import json
import os

generator = pipeline(
    "text-generation",
    model="kakaocorp/kanana-nano-2.1b-instruct",
    device=0
)

prompt_path = "prompts/v21_prompt.jsonl"
result_path = "results/v21_result.md"
os.makedirs(os.path.dirname(result_path), exist_ok=True)

with open(prompt_path, "r", encoding="utf-8") as f:
    prompt = json.loads(f.readline())

situation_text = (
    f"[내적설명]\n" + "\n".join(prompt['inner_monologue'][:3]) + "\n\n"
    f"[시스템]\n" + "\n".join(prompt['system_msgs'][:3]) + "\n\n"
)

sample_dialogues = prompt["dialogue"][:5]
dialogue_text = "\n".join(f"- {line}" for line in sample_dialogues)
if len(prompt["dialogue"]) > 5:
    dialogue_text += f"\n... (총 {len(prompt['dialogue'])}개 중 일부 예시)"


choices_text = "\n".join(prompt["choices"])

input_text = (
    f"{situation_text}"
    f"[참고 대사 예시]\n{dialogue_text}\n\n"
    f"[선택지]\n{choices_text}\n\n"
    f"{prompt['question']}\n\n"
    "※ 응답 형식 예시\n"
    "- 선택지 1-1: ...\n  - 성진우의 대사: \"...\"\n  - 선택 이유: ...\n"
    "- 선택지 1-2: ...\n  - 성진우의 대사: \"...\"\n  - 선택 이유: ...\n"
    "- 선택지 2: ...\n  - 성진우의 대사: \"...\"\n  - 선택 이유: ...\n"
    "- 선택지 3-1: ...\n  - 성진우의 대사: \"...\"\n  - 선택 이유: ...\n"
    "- 선택지 3-2: ...\n  - 성진우의 대사: \"...\"\n  - 선택 이유: ...\n"
    "\n[최종 선택]\n- 선택 번호: 반드시 (1-1, 1-2, 2, 3-1, 3-2 중 하나)\n"
    "- 이유: 위 비교를 바탕으로 선택 근거를 명확히 작성.\n"
)

outputs = generator(
    input_text,
    max_new_tokens=600,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.92
)

output_text = outputs[0]["generated_text"]
trimmed_output = output_text.replace(input_text, "").strip()

with open(result_path, "w", encoding="utf-8") as f:
    f.write(f"# v21 출력 결과\n\n")
    f.write(f"```\n{trimmed_output}\n```\n")

print(f"v21 결과 저장 완료 → {result_path}")


Device set to use cuda:0


v21 결과 저장 완료 → results/v21_result.md
